Integrantes:

*   Daniel Perea Mercado
*   David Diaz Rodriguez
*   Nicolas Niño Valderrama
*   Valentina Jimenez Torres

___
# Instalación de librerias

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

from scipy.cluster.hierarchy import linkage, fcluster, cophenet, dendrogram
from scipy.stats import zscore, shapiro, kstest, anderson
from scipy.spatial.distance import pdist

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score,  calinski_harabasz_score
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [ ]:
!pip install kneed

In [ ]:
from kneed import KneeLocator

In [ ]:
# Configuramos el cuadernos para que los resultados contenga
# solo 3 cifras decimales
#pd.options.display.float_format = '{:.3f}'.format

---
# **Lectura de los datos desde Kaggle**

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!echo '{"username":"nicolsniovalderrama","key":"f7f9fda0ddecb368ca50593318da1412"}' > ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d devantltd/analysing-second-hand-car-sales-data

In [ ]:
!unzip analysing-second-hand-car-sales-data

In [ ]:
df_shc = pd.read_csv("second_hand_car_sales.csv")
df_shc.head()

___
# Exploración de los datos

In [ ]:
df_shc.info()

In [ ]:
df_shc.shape

In [ ]:
df_shc.dtypes

## Datos nulos

In [ ]:
df_shc.isnull().sum()

El dataframe no presenta datos nulos

## Datos duplicados

In [ ]:
df_shc.duplicated().sum()

No hay datos duplicados

## Análisis variables númericas

#### Distribuciones

In [ ]:
numericas = df_shc.select_dtypes(include=[np.number])
numericas.describe()

Al analizar los datos de la tabla podemos notar que los datos se comportan con normalidad, *Engine size* esta expresada en litros y hace referencia la cilindrada o a la suma del volumen útil de todos los cilindros del motor, el *Year of Manufacturer* indica el año en que el auto fue ensamblado, el *Mileage* indica cuantas millas ha recorrido el vechiculo y *Price*, expreado en libras esternilas, expresa el valor de mercado del automovil.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))  # 2 filas y 2 columnas
axs = axs.flatten()  # Aplanar el array de ejes para facilitar el acceso

# Iterar sobre las columnas numéricas y graficar
for i, col in enumerate(numericas.columns):
    sns.histplot(df_shc[col], kde=True, ax=axs[i], color='skyblue', edgecolor='black')
    axs[i].set_title(f'Distribución de {col}')
    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Frecuencia')

# Ajustar el layout
plt.tight_layout()
plt.show()

Las cuatro variables analizadas (*Engine Size (L), Year of Manufacture, Mileage, Price (£)*) presentan distribuciones bastante uniformes, lo que indica una variedad de datos sin sesgos evidentes hacia valores específicos.

In [ ]:
numericas

##### Prueba de Shapiro-Wilk

Se realiza en este caso la prueba de normalidad con diferentes grados de signifcancia

In [ ]:
# Aplicar la prueba de Shapiro-Wilk a cada variable
for variable in numericas:
    print(f'\nPrueba de Shapiro-Wilk para {variable}:')
    stat, p_value = shapiro(numericas[variable])
    print('Estadístico=%.3f, p-valor=%.6f' % (stat, p_value))
    if p_value > 0.05:
        print('La variable sigue una distribución normal')
    else:
        print('La variable no sigue una distribución normal')

##### Prueba de Kolmogorov-Smirnov (KS)

In [ ]:
# Prueba de Kolmogorov-Smirnov
for variable in numericas:
    print(f'\nPrueba de Kolmogorov-Smirnov para {variable}:')
    stat, p_value = kstest(numericas[variable], 'norm')
    print('Estadístico=%.3f, p-valor=%.6f' % (stat, p_value))
    if p_value > 0.05:
        print('La variable sigue una distribución normal')
    else:
        print('La variable no sigue una distribución normal')

##### Prueba de Anderson-Darling

In [ ]:
# Prueba de Anderson-Darling
for variable in numericas:
    print(f'\nPrueba de Anderson-Darling para {variable}:')
    result = anderson(numericas[variable], dist='norm')
    print('Estadístico=%.3f' % result.statistic)
    for i in range(len(result.critical_values)):
        print('Nivel de significancia=%.1f%%: estadístico crítico=%.3f' % (result.significance_level[i], result.critical_values[i]))
        if result.statistic < result.critical_values[i]:
            print('La variable sigue una distribución normal')
        else:
            print('La variable no sigue una distribución normal')

Las pruebas de normalidad de Shapiro, Kolmogorov y Anderson indican que estas variables no siguen una distribución normal.

#### Valores atípicos

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))  # 2 filas y 2 columnas

axes = axes.flatten()

# Crear un boxplot para cada variable numérica
for ax, col in zip(axes, numericas.columns):
    sns.boxplot(y=df_shc[col], ax=ax, color='skyblue')
    ax.set_title(f'Boxplot de {col}')
    ax.set_ylabel(col)

# Ajustar el layout para evitar superposiciones
plt.tight_layout()
plt.show()

In [ ]:
for col in numericas.columns:
    # Calcular z-score para cada columna y obtener su valor absoluto
    z_scores = zscore(df_shc[col])
    abs_z_scores = np.abs(z_scores)

    # Seleccionar los outliers usando un límite de 3
    outliers_zscore = df_shc[abs_z_scores > 3]

    # Contar el número de valores atípicos
    num_outliers = outliers_zscore[col].count()

    # Imprimir el nombre de la columna, el valor mínimo del outlier y el número de outliers
    print(f"Variable: {col}")
    print(f"Número de valores atípicos: {num_outliers}")
    if num_outliers > 0:  # Solo imprimir el valor mínimo si hay outliers
        print(f"Outlier mínimo (z-score): {outliers_zscore[col].min()}")
    print("---")

Aunque las variables no siguen una distribución normal, la falta de valores atípicos refuerza la calidad y la confiabilidad de los datos. Esto indica que los datos son consistentes y homogéneos dentro de los rangos esperados, lo cual es positivo para el análisis, ya que no se requieren medidas adicionales para manejar valores extremos.

#### Correlaciones

In [ ]:
#Matriz de correlación
matriz_correlacion = numericas.corr()

print("Matriz de correlación entre las variables numéricas:")
print(matriz_correlacion)

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))

# Mostrar la matriz de correlación como mapa de calor
sns.heatmap(matriz_correlacion, vmax=.8, square=True, annot=True, annot_kws={"fontsize": 10}, fmt=".2f", cmap='Blues')

# Agregar título y etiquetas
plt.title('Matriz de Correlación')
plt.xlabel('Variables')
plt.ylabel('Variables')

# Mostrar la figura
plt.show()

La matriz de correlación entre las variables numéricas indica que no hay correlaciones significativas entre las variables analizadas.

## Análisis variables categóricas

#### Distribuciones

In [ ]:
variables = ['Manufacturer', 'Model', 'Fuel Type']

for var in variables:
    print(f"\n-------------------\nFrecuencia de la variable '{var}':")
    frecuencia = df_shc[var].value_counts()
    for categoria, freq in frecuencia.items():
        print(f"{categoria} {freq}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # 1 fila, 3 columnas

for ax, var in zip(axes, variables):
    frecuencia = df_shc[var].value_counts()

    # Graficar la frecuencia con Matplotlib
    frecuencia.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
    ax.set_title(f'{var}')
    ax.set_xlabel('Categoría')
    ax.set_ylabel('Frecuencia')
    ax.set_xticklabels(frecuencia.index, rotation=45)  # Rotar etiquetas para mejor visibilidad

# Ajustar el layout para evitar superposiciones
plt.tight_layout()
plt.show()

## Tratamiento de los datos

___

Se decidió trabajar sólo con las variables numéricas del dataframe ya que son las relevantes para el tipo de modelos que vamos a trabajar, así que se eliminan las variables 'Manufacturer', 'Model', 'Fuel Type'.

In [ ]:
# Eliminar columnas categóricas específicas
df_shc2 = df_shc.drop(columns=['Manufacturer', 'Model', 'Fuel Type'])

# Mostrar las primeras filas del DataFrame resultante
df_shc2.head()

En el caso de la variable *Year of Manufacture*, para poder darle un mejor tratamiento como número, se ha convertido a *Vehicle Age* que hace referencia a la edad, siendo esta la diferencia entre el año actual 2024 y el año de su manufactura.

In [ ]:
df_shc2['Vehicle Age'] = 2024 - df_shc2['Year of Manufacture']
df_shc2 = df_shc2.drop(columns=['Year of Manufacture'])

In [ ]:
df_shc2.head()

# Modelos

## K - Means

Se tiene en cuenta:
- Sensible a las escalas de los valores
- Sensible a correlaciones altas entre variables
- Asume cúmulos esféricos
- Atípicos sesgan resultados
- Hiperparámetro número de clusters (se optimiza con método del codo y coeficiente de silueta)

### Tratamiento para el modelo

In [ ]:
# Copia de la base de datos con transformaciones
df_shc3 = df_shc2.copy()
df_shc3.head()

In [ ]:
# Escalamos el dataframe
df_shc3 = pd.DataFrame(StandardScaler().fit_transform(df_shc3), columns=list(df_shc3.columns))

df_shc3.head()

In [ ]:
# Para la implementación del modelo tomamos una muestra aleatoria del dataframe de 15.000 datos
data_aleatorio1 = df_shc3.sample(n=15000, random_state=123)
data_aleatorio1.head()

In [ ]:
# Preparamos copias de la muestra aleatoria para probar en diferentes modelos
data_aleatorio2 = data_aleatorio1.copy()
data_aleatorio3 = data_aleatorio1.copy()
data_aleatorio4 = data_aleatorio1.copy()
data_aleatorio5 = data_aleatorio1.copy()
data_aleatorio6 = data_aleatorio1.copy()
data_aleatorio7 = data_aleatorio1.copy()

### Optmización del número de cluster

#### Método del codo

In [ ]:
# Número optimo de K según método del codo
inertia_list = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=113)
    kmeans.fit(data_aleatorio1)
    inertia_list.append(kmeans.inertia_)

plt.plot(range(1,11),inertia_list)
plt.scatter(range(1,11),inertia_list)
plt.xlabel("Número de clusters", size=10)
plt.ylabel("Valor de inercia", size=10)
plt.title("Valores de inercia según número de clusters", size=12)
plt.show()

In [ ]:
# Seleccion automatica del numero k
kl = KneeLocator(range(1, 11),
                 inertia_list, curve="convex",
                 direction="decreasing")
kl.elbow

El análisis automatizado del punto de codo utilizando *KneeLocator* ha determinado que el número óptimo de clusters es 4.

#### Método de la silueta

In [ ]:
# Número optimo de K según método de la silueta
silhouette_scores = []

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=123)
    kmeans.fit(data_aleatorio1)
    score_s = silhouette_score(data_aleatorio1, kmeans.labels_)
    silhouette_scores.append(score_s)

plt.plot(range(2, 11), silhouette_scores)
plt.xticks(range(2,11))
plt.xlabel("Número de clusters")
plt.ylabel("Coeficiente de Silueta")
plt.title("Coeficiente de silueta vs número de clusters")
plt.show()

Graficamente el método de la silueta nos sugiere un número de cluster de 8.

### Modelos KMeans sin reducción de dimensionalidad

#### Segun codo (4 clusters)

In [ ]:
kmeans_constants = {"init": "k-means++", "n_init": 100, "max_iter": 500, "random_state": 123}

# --- Modelo K-means ---
model_kmeans_4 = KMeans(n_clusters = 4, **kmeans_constants)
model_kmeans_4.fit(data_aleatorio1)

In [ ]:
# Configuración de estilo con colores vivos y más contraste
sns.set(style="whitegrid", context="notebook", palette="bright")

# Aplicar PCA para reducir a 2 componentes principales
pca = PCA(n_components=2)
df_PCA = pca.fit_transform(data_aleatorio1)

# Predecir los clústeres para los datos transformados por PCA
labels = model_kmeans_4.predict(data_aleatorio1)

# Crear el gráfico de dispersión con colores llamativos
plt.figure(figsize=(12, 12))
scatter = plt.scatter(df_PCA[:, 0], df_PCA[:, 1], c=labels, s=50, cmap="Set1", alpha=0.9, edgecolor='black', linewidth=0.6)

# Configuración del título y etiquetas de los ejes
plt.title("Visualización de Clústeres - K-Means (4 Clústeres)", fontsize=15, fontweight='bold', color='black')
plt.xlabel("Componente Principal 1", fontsize=10, fontweight='bold')
plt.ylabel("Componente Principal 2", fontsize=10, fontweight='bold')

# Agregar una leyenda para los clústeres
legend = plt.legend(*scatter.legend_elements(), title="Clústeres", fontsize=14, title_fontsize='15', loc="upper right", fancybox=True, shadow=True)
plt.setp(legend.get_title(), fontweight='bold')

# Mejorar la estética con un fondo semi-transparente y ajustes de cuadrícula
plt.grid(True, linestyle='--', alpha=0.6)
plt.gca().patch.set_alpha(0.95)  # Fondo semi-transparente
plt.tight_layout()

# Mostrar el gráfico
plt.show()

In [ ]:
# --- Evaluación del modelo kmeans ---
print('Inertia: ', model_kmeans_4.inertia_)
print("Silhouette Score: ", silhouette_score(data_aleatorio1, model_kmeans_4.labels_))
print("Calinski Harabasz Score: ", calinski_harabasz_score(data_aleatorio1, model_kmeans_4.labels_))

In [ ]:
resultados_kmeans = pd.DataFrame(columns=['Modelo', 'Clusters', 'Inercia', 'Silhouette Score', 'Calinski-Harabasz'])

In [ ]:
# Recopila los resultados en un diccionario
resultados = {
    'Modelo': 'KMeans sin reducción',
    'Clusters': model_kmeans_4.n_clusters,
    'Inercia': model_kmeans_4.inertia_,
    'Silhouette Score': silhouette_score(data_aleatorio1, model_kmeans_4.labels_),
    'Calinski-Harabasz': calinski_harabasz_score(data_aleatorio1, model_kmeans_4.labels_)
}

# Agrega los resultados al DataFrame
resultados_kmeans = pd.concat([resultados_kmeans, pd.DataFrame([resultados])], ignore_index=True)

resultados_kmeans.head(10)

#### Segun silueta (8 clusters)

In [ ]:
kmeans_constants = {"init": "k-means++", "n_init": 100, "max_iter": 500, "random_state": 123}

# --- Modelo K-means ---
model_kmeans_8 = KMeans(n_clusters = 8, **kmeans_constants)
model_kmeans_8.fit(data_aleatorio2)

In [ ]:
# Configuración de estilo con colores vivos y más contraste
sns.set(style="whitegrid", context="notebook", palette="bright")

# Aplicar PCA para reducir a 2 componentes principales
pca = PCA(n_components=2)
df_PCA = pca.fit_transform(data_aleatorio2)

# Predecir los clústeres para los datos transformados por PCA
labels = model_kmeans_8.predict(data_aleatorio2)

# Crear el gráfico de dispersión con colores llamativos
plt.figure(figsize=(12, 12))
scatter = plt.scatter(df_PCA[:, 0], df_PCA[:, 1], c=labels, s=50, cmap="Set1", alpha=0.9, edgecolor='black', linewidth=0.6)

# Configuración del título y etiquetas de los ejes
plt.title("Visualización de Clústeres - K-Means (8 Clústeres)", fontsize=15, fontweight='bold', color='black')
plt.xlabel("Componente Principal 1", fontsize=10, fontweight='bold')
plt.ylabel("Componente Principal 2", fontsize=10, fontweight='bold')

# Agregar una leyenda para los clústeres
legend = plt.legend(*scatter.legend_elements(), title="Clústeres", fontsize=14, title_fontsize='15', loc="upper right", fancybox=True, shadow=True)
plt.setp(legend.get_title(), fontweight='bold')

# Mejorar la estética con un fondo semi-transparente y ajustes de cuadrícula
plt.grid(True, linestyle='--', alpha=0.6)
plt.gca().patch.set_alpha(0.95)  # Fondo semi-transparente
plt.tight_layout()

# Mostrar el gráfico
plt.show()

In [ ]:
# --- Evaluación del modelo kmeans ---
print('Inertia: ', model_kmeans_8.inertia_)
print("Silhouette Score: ", silhouette_score(data_aleatorio2, model_kmeans_8.labels_))
print("Calinski Harabasz Score: ", calinski_harabasz_score(data_aleatorio2, model_kmeans_8.labels_))

In [ ]:
# Recopila los resultados en un diccionario
resultados = {
    'Modelo': 'KMeans sin reducción',
    'Clusters': model_kmeans_8.n_clusters,
    'Inercia': model_kmeans_8.inertia_,
    'Silhouette Score': silhouette_score(data_aleatorio2, model_kmeans_8.labels_),
    'Calinski-Harabasz': calinski_harabasz_score(data_aleatorio2, model_kmeans_8.labels_)
}

# Agrega los resultados al DataFrame
resultados_kmeans = pd.concat([resultados_kmeans, pd.DataFrame([resultados])], ignore_index=True)

resultados_kmeans.head(10)

### Reducción de dimensionalidad

In [ ]:
pca = PCA(n_components=0.90).fit(data_aleatorio1)
pca.n_components_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# --- Varianza explicada ---
PCA_variance = pd.DataFrame({'Varianza explicada (%)':
                             pca.explained_variance_ratio_*100})

fig, ax = plt.subplots(1, 1, figsize = (7, 5))

bar = sns.barplot(x = ['PC ' + str(i) for i in range(1, 5)],
                  y = PCA_variance['Varianza explicada (%)'],
                  linewidth = 1.5, edgecolor = 'k', color = '#4bafb8',
                  alpha = 0.8)

plt.show()

In [ ]:
pca = PCA(n_components=3).fit(data_aleatorio1)

pca.n_components_

X_pca = pca.fit_transform(data_aleatorio1)
X_pca[:3]

In [ ]:
X_pca2 = X_pca.copy()
X_pca3 = X_pca.copy()

In [ ]:
pesos_pca = pd.DataFrame(pca.components_, columns = data_aleatorio1.columns,
             index = ['PC 1', 'PC 2', 'PC 3']).round(2).T

pesos_pca

#### Nuevo método codo

In [ ]:
# Número optimo de K según método del codo
inertia_list_pca = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=113)
    kmeans.fit(X_pca)
    inertia_list_pca.append(kmeans.inertia_)

plt.plot(range(1,11),inertia_list_pca)
plt.scatter(range(1,11),inertia_list_pca)
plt.xlabel("Número de clusters", size=10)
plt.ylabel("Valor de inercia", size=10)
plt.title("Valores de inercia según número de clusters", size=12)
plt.show()

In [ ]:
# Seleccion automatica del numero k
kl = KneeLocator(range(1, 11),
                 inertia_list_pca, curve="convex",
                 direction="decreasing")
kl.elbow

#### Nuevo método silueta

In [ ]:
silhouette_scores_pca = []

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=3)
    kmeans.fit(X_pca)
    score_s = silhouette_score(X_pca, kmeans.labels_)
    silhouette_scores_pca.append(score_s)

# Visualización de score de silueta vs k
plt.plot(range(2, 11), silhouette_scores_pca)
plt.xticks(range(2,11))
plt.xlabel("Número de clusters")
plt.ylabel("Coeficiente de Silueta")
plt.title("Coeficiente de silueta vs número de clusters")
plt.show()

Graficamente el método de la silueta nos sugiere un número de cluster de 6.

### Modelos KMeans con reducción de dimensión

#### Nuevo kmeans según codo (4 clusters)

In [ ]:
kmeans_constants = {"init": "k-means++", "n_init": 100, "max_iter": 500, "random_state": 123}

# --- Modelo K-means ---
model_kmeans_pca_4 = KMeans(n_clusters = 4, **kmeans_constants)
model_kmeans_pca_4.fit(X_pca)

In [ ]:
X_pca = PCA(n_components=2).fit_transform(X_pca)
model_kmeans_pca_4 = KMeans(n_clusters=4, init="k-means++", n_init=100, max_iter=500, random_state=123).fit(X_pca)
labels = model_kmeans_pca_4.labels_

# Configuración de estilo y gráfico
sns.set(style="whitegrid", context="notebook", palette="tab10")
plt.figure(figsize=(12, 12))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, s=50, cmap="tab10", alpha=0.9, edgecolor='black', linewidth=0.6)

# Títulos, etiquetas y leyenda
plt.title("Visualización de Clústeres - K-Means PCA (4 Clústeres)", fontsize=15, fontweight='bold')
plt.xlabel("Componente Principal 1", fontsize=10, fontweight='bold')
plt.ylabel("Componente Principal 2", fontsize=10, fontweight='bold')
plt.legend(*scatter.legend_elements(), title="Clústeres", fontsize=14, title_fontsize='15', loc="upper right", fancybox=True, shadow=True)

# Mejorar estética y mostrar gráfico
plt.grid(True, linestyle='--', alpha=0.6)
plt.gca().patch.set_alpha(0.95)
plt.tight_layout()
plt.show()

In [ ]:
# --- Evaluación del modelo kmeans con dataset reducido ---
print(" ### K-MEANS ###")
print('Inertia: ', model_kmeans_pca_4.inertia_)
print('Silhouette Score: ', silhouette_score(X_pca, model_kmeans_pca_4.labels_))
print('Calinski harabasz score: ', calinski_harabasz_score(X_pca, model_kmeans_pca_4.labels_))

In [ ]:
# Recopila los resultados en un diccionario
resultados = {
    'Modelo': 'KMeans con PCA',
    'Clusters': model_kmeans_pca_4.n_clusters,
    'Inercia': model_kmeans_pca_4.inertia_,
    'Silhouette Score': silhouette_score(X_pca, model_kmeans_pca_4.labels_),
    'Calinski-Harabasz': calinski_harabasz_score(X_pca, model_kmeans_pca_4.labels_)
}

# Agrega los resultados al DataFrame
resultados_kmeans = pd.concat([resultados_kmeans, pd.DataFrame([resultados])], ignore_index=True)

# Muestra la tabla actualizada
resultados_kmeans.head(10)

#### Nuevo kmeans según silueta (6 clusters)

In [ ]:
kmeans_constants = {"init": "k-means++", "n_init": 100, "max_iter": 500, "random_state": 123}

# --- Modelo K-means ---
model_kmeans_pca_6 = KMeans(n_clusters = 6, **kmeans_constants)
model_kmeans_pca_6.fit(X_pca2)

In [ ]:
X_pca2 = PCA(n_components=2).fit_transform(X_pca2)
model_kmeans_pca_6 = KMeans(n_clusters=6, init="k-means++", n_init=100, max_iter=500, random_state=123).fit(X_pca2)
labels = model_kmeans_pca_6.labels_

# Configuración de estilo y gráfico
sns.set(style="whitegrid", context="notebook", palette="tab10")
plt.figure(figsize=(12, 12))
scatter = plt.scatter(X_pca2[:, 0], X_pca2[:, 1], c=labels, s=50, cmap="tab10", alpha=0.9, edgecolor='black', linewidth=0.6)

# Títulos, etiquetas y leyenda
plt.title("Visualización de Clústeres - K-Means PCA (6 Clústeres)", fontsize=15, fontweight='bold')
plt.xlabel("Componente Principal 1", fontsize=10, fontweight='bold')
plt.ylabel("Componente Principal 2", fontsize=10, fontweight='bold')
plt.legend(*scatter.legend_elements(), title="Clústeres", fontsize=14, title_fontsize='15', loc="upper right", fancybox=True, shadow=True)

# Mejorar estética y mostrar gráfico
plt.grid(True, linestyle='--', alpha=0.6)
plt.gca().patch.set_alpha(0.95)
plt.tight_layout()
plt.show()

In [ ]:
# --- Evaluación del modelo kmeans con dataset reducido ---
print(" ### K-MEANS ###")
print('Inertia: ', model_kmeans_pca_6.inertia_)
print('Silhouette Score: ', silhouette_score(X_pca2, model_kmeans_pca_6.labels_))
print('Calinski harabasz score: ', calinski_harabasz_score(X_pca2, model_kmeans_pca_6.labels_))

In [ ]:
# Recopila los resultados en un diccionario
resultados = {
    'Modelo': 'KMeans con PCA',
    'Clusters': model_kmeans_pca_6.n_clusters,
    'Inercia': model_kmeans_pca_6.inertia_,
    'Silhouette Score': silhouette_score(X_pca2, model_kmeans_pca_6.labels_),
    'Calinski-Harabasz': calinski_harabasz_score(X_pca2, model_kmeans_pca_6.labels_)
}

# Agrega los resultados al DataFrame
resultados_kmeans = pd.concat([resultados_kmeans, pd.DataFrame([resultados])], ignore_index=True)

# Muestra la tabla actualizada
resultados_kmeans.head(10)

## Jerárquico

- Dependiendo del cálculo de proximidades puede ser sensible al ruido y a atípicos y tiende a romper grupos grandes
- Utiliza formas convexas.
- Se debe definir la forma de calcular las proximidades
    - Max (Funciona bien con ruido entre grupos, está sesgado a formas circulares y tiende a romper grupos grandes)
    - Promedio grupo (Funciona bien con ruido entre grupos y está sesgado a formas circulares)
    - Ward (es común, funciona igual que promedio grupo pero con SSE, es poco susceptible al ruido y a los datos atípicos)


### Distribución de la muestra

Comprobemos la distribución de la muestra

In [ ]:
data = df_shc3.sample(n=3000, random_state=123)
numericas1 = data.select_dtypes(include=[np.number])

In [ ]:
# Escalamos el dataframe
numericas1  = pd.DataFrame(StandardScaler().fit_transform(numericas1 ), columns=list(numericas1 .columns))
numericas1 .head()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))  # 2 filas y 2 columnas
axs = axs.flatten()  # Aplanar el array de ejes para facilitar el acceso

# Iterar sobre las columnas numéricas y graficar
for i, col in enumerate(numericas1.columns):
    sns.histplot(data[col], kde=True, ax=axs[i], color='skyblue', edgecolor='black')
    axs[i].set_title(f'Distribución de {col}')
    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Frecuencia')

# Ajustar el layout
plt.tight_layout()
plt.show()

Luego de comparar la distribución de una muestra del 6% de los datos originales con la distribución de estos mismos datos, concluimos que ambas son similares. Esto nos permite afirmar que la muestra del 6% será adecuada para entrenar el modelo, lo que nos ayudará a ahorrar capacidad computacional.

### Dendograma

In [ ]:
# Lista de métodos de linkage a probar
methods = ['single', 'complete', 'average', 'ward']
label_list = list(data['Vehicle Age'])

plt.figure(figsize=(25, 18))

# Generar y graficar un dendrograma por cada método de linkage
for i, method in enumerate(methods, 1):
    Z = linkage(data, method=method)

    # Subplot para cada dendrograma
    plt.subplot(2, 2, i)
    plt.title(f'Dendrograma - Método: {method}')
    dendrogram(
            Z,
            labels=label_list,
            leaf_rotation=90,
            distance_sort = 'descending',
            orientation='right',
            show_leaf_counts=False,
            leaf_font_size=14)
    plt.xlabel('Índices de las muestras')
    plt.ylabel('Distancia')
plt.tight_layout()
plt.show()


- **Método "Single":** Muy sensible al ruido, el dendrograma se ve muy denso en las primeras fusiones, lo que sugiere que hay muchas fusiones pequeñas a distancias muy cortas y no genera agrupación de clústeres grandes.
- **Método "Complete":** Hay una distancia clara y significativa entre las últimas fusiones, lo que puede ser útil para decidir un corte en el dendrograma. Los clústeres resultantes tienden a estar más equilibrados.
- **Método "Average":** El dendrograma muestra un comportamiento intermedio entre los métodos "Single" y "Complete". Los clústeres no están tan fragmentados como en el método "Single", ni tan compactos como en "Complete".
- **Método "Ward":** Se observa que los clústeres se forman de manera muy jerárquica, con claras divisiones en niveles de distancia bien definidos.
Hay pocas fusiones grandes al final, lo que sugiere que los clústeres son bien separados y pueden ser más fáciles de interpretar.

Debido a estas conclusiones hemos definido que usaremos el método "Ward"

In [ ]:
 #Funcion de vinculación para agrupar los datos según similitud

 Z = linkage(data, 'ward')

 # Gráfica del dendrograma
 label_list = list(data['Vehicle Age'])

 plt.figure(figsize=(25, 18))
 dendrogram(
     Z,
     labels=label_list,
     leaf_rotation=90,
     distance_sort = 'ascending',
     orientation='right',
     show_leaf_counts=False,
     leaf_font_size=14)
 plt.show()

Realizaremos una comparación visual, entre el coeficiente de silueta y de Calinski con diferentes números de clústeres, para finalmente tomar la desición de la cantidad de clústeres.

In [ ]:
# Para almacenar resultados
silhouette_scores_hc = []
calinski_scores_hc = []

# Recorreremos el número de clusters de 2 a 10
for k in range(2, 11):
    # Instanciar AgglomerativeClustering con el número de clústeres
    hc_t1 = AgglomerativeClustering(n_clusters=k,
                                 linkage='ward')
    hc_t1.fit(data)

    # Calcular el coeficiente de silueta
    score_s = silhouette_score(data, hc_t1.labels_)
    silhouette_scores_hc.append(score_s)

    # Calcular el coeficiente de Calinski-Harabasz
    score_c = calinski_harabasz_score(data, hc_t1.labels_)
    calinski_scores_hc.append(score_c)

# Visualizar los resultados
import matplotlib.pyplot as plt

x_range = range(2, 11)
plt.figure(figsize=(10, 5))

# Gráfico del coeficiente de silueta
plt.subplot(1, 2, 1)
plt.plot(x_range, silhouette_scores_hc, marker='o', label='Silhouette Score')
plt.title('Coeficiente de Silueta')
plt.xlabel('Número de Clústeres')
plt.ylabel('Puntuación Silueta')
plt.grid(True)

# Gráfico del coeficiente de Calinski-Harabasz
plt.subplot(1, 2, 2)
plt.plot(x_range, calinski_scores_hc, marker='o', label='Calinski-Harabasz Score', color='orange')
plt.title('Coeficiente de Calinski-Harabasz')
plt.xlabel('Número de Clústeres')
plt.ylabel('Puntuación Calinski-Harabasz')
plt.grid(True)

plt.tight_layout()
plt.show()

El número de clústeres que nos dan la "mejor" combinación entre el coeficiente de Silueta y de Calinski-Harabasz es 3. Debido a que la combinación que arroja 6 clústeres da una mayor perdida en el coeficiente de Calinski que ganancia en el de Silueta y una mayor cantidad de clústeres podria no ser de ayuda.

### Entrenamiento del modelo de agrupación Jerárquica en el conjunto de datos

In [ ]:
hc = AgglomerativeClustering(
    n_clusters = 3,
    linkage = 'ward'
)

y_hc = hc.fit_predict(data)
print(y_hc)

In [ ]:
data.loc[:, 'Cluster'] = y_hc
data.head()

### Visualización con PCA

In [ ]:
# Reducir a 3 dimensiones con PCA
pca = PCA(n_components=3)
data_3d = pca.fit_transform(data)

# Aplicar clustering a los datos en 3D
y_hc = hc.fit_predict(data_3d)

# Crear la figura 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Visualizar los clusters en 3D
u_labels = np.unique(y_hc)
for i in u_labels:
    ax.scatter(
        data_3d[y_hc == i, 0],  # Primera componente principal
        data_3d[y_hc == i, 1],  # Segunda componente principal
        data_3d[y_hc == i, 2],  # Tercera componente principal
        label=f'Cluster {i}'
    )

# Configurar etiquetas y leyenda
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('PCA 3')
plt.title('3D Visualization of Clusters with PCA')
plt.legend()
plt.show()

In [ ]:
# Reducir a 2 dimensiones con PCA
pca = PCA(n_components=2)
data_3d = pca.fit_transform(data)

# Aplicar clustering a los datos en 3D
y_hc = hc.fit_predict(data_3d)

# Crear la figura 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Visualizar los clusters en 3D
u_labels = np.unique(y_hc)
for i in u_labels:
    ax.scatter(
        data_3d[y_hc == i, 0],  # Primera componente principal
        data_3d[y_hc == i, 1],  # Segunda componente principal
        label=f'Cluster {i}'
    )

# Configurar etiquetas y leyenda
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
plt.title('3D Visualization of Clusters with PCA')
plt.legend()
plt.show()

## Gaussian Mixture Clustering

- Supone una distribución normal en las características
- Sensible a atípicos
- No asume cúmulos esféricos
- Agrupa los datos que pertenecen a cierta distribución
- Menos sensible a las diferencias en escalas
- Realizar multiples iteraciones para la inicialización como k means
- Se necesita definir el número de clusters:
    - AIC
    - BIC
- Hiperparametro tipo de matriz de covarianza


### Reducción de dimensionalidad

In [ ]:
pca_gmm = PCA(n_components=0.90).fit(data_aleatorio5)
pca_gmm .n_components_

In [ ]:
pca_gmm.explained_variance_ratio_

In [ ]:
pca_gmm = PCA(n_components=3).fit(data_aleatorio5)
X_pca_gmm = pca_gmm.fit_transform(data_aleatorio5)

In [ ]:
X_pca_gmm2 = X_pca_gmm.copy()

### Estimación de las métricas BIC y AIC

In [ ]:
# Estimación de hiperparámetros, con las metricas BIC, AIC
# Se estima un numero de componentes aleatorio haciendo una variación de 2 a 12
n_components = np.arange(2, 12)

models_g = [GaussianMixture(n_components=n, random_state=123).fit(X_pca_gmm) for n in n_components]

plt.plot(n_components, [m.bic(X_pca_gmm) for m in models_g], label="BIC")
plt.plot(n_components, [m.aic(X_pca_gmm) for m in models_g], label="AIC")
plt.legend()
plt.xlabel("Número de Clusters")

Podemos observas en a gráfica que para ambas metricas difieren un poco pero, ambas muestran que está entre 4 y 5 clusters que podríamos analizar

### Modelo con 4 clusters

In [ ]:
# Modelo GMM- entrenamiento del modelo con 4 componentes con una covarianza tipo full
model_gmm_4 = GaussianMixture(n_components=4, random_state=123, covariance_type = 'full').fit(X_pca_gmm)

In [ ]:
# Predicciones y evaluación del modelo GMM con las metricas silhouette_score y calinski
labels_ = model_gmm_4.predict(X_pca_gmm)

print(" ### GMMM ###")
print('Silhouette Score: ', silhouette_score(X_pca_gmm, labels_))
print('Calinski harabasz score: ', calinski_harabasz_score(X_pca_gmm, labels_))

In [ ]:
clusters_4 = model_gmm_4.predict(X_pca_gmm)
data_aleatorio5['clusters_4'] = clusters_4

# Predicciones de los cluster
data_aleatorio5['PCA_1'] = X_pca_gmm[:, 0]
data_aleatorio5['PCA_2'] = X_pca_gmm[:, 1]
data_aleatorio5['PCA_3'] = X_pca_gmm[:, 2]

# 3d scatterplot using plotly.express
data_aleatorio5['clusters_4'] = data_aleatorio5['clusters_4'].astype('category')
fig = px.scatter_3d(data_aleatorio5, x="PCA_1", y="PCA_2", z="PCA_3", color="clusters_4")
fig.update_layout(width=1000, height=700)
fig.show()

### Modelo con 5 clusters

Ahora con 5 componentes

In [ ]:
# Modelo GMM- entrenamiento del modelo con 5 componentes con una covarianza tipo full
model_gmm_5 = GaussianMixture(n_components=5, random_state=123, covariance_type = 'full').fit(X_pca_gmm2)

In [ ]:
# Predicciones y evaluación del modelo GMM con las metricas silhouette_score y calinski
labels_ = model_gmm_5.predict(X_pca_gmm2)

print(" ### GMMM ###")
print('Silhouette Score: ', silhouette_score(X_pca_gmm2, labels_))
print('Calinski harabasz score: ', calinski_harabasz_score(X_pca_gmm2, labels_))

In [ ]:
clusters_5 = model_gmm_5.predict(X_pca_gmm)
data_aleatorio6['clusters_5'] = clusters_5

# Predicciones de los cluster
data_aleatorio6['PCA_1'] = X_pca_gmm2[:, 0]
data_aleatorio6['PCA_2'] = X_pca_gmm2[:, 1]
data_aleatorio6['PCA_3'] = X_pca_gmm2[:, 2]

# 3d scatterplot using plotly.express
data_aleatorio6['clusters_5'] = data_aleatorio6['clusters_5'].astype('category')
fig = px.scatter_3d(data_aleatorio6, x="PCA_1", y="PCA_2", z="PCA_3", color="clusters_5")
fig.update_layout(width=1000, height=700)
fig.show()

### Transformaciones para normalidad

In [ ]:
from scipy import stats
data_transformed = data_aleatorio7.copy()

# Función para aplicar transformación logarítmica (evitar log(0) con un pequeño offset)
def log_transform(column):
    return np.log1p(column)

# Función para aplicar transformación de raíz cuadrada
def sqrt_transform(column):
    return np.sqrt(column)

# Aplicar transformación de Box-Cox (solo para valores positivos)
def boxcox_transform(column):
    column_positive = column + 1e-6  # Evitar valores <= 0
    transformed, _ = stats.boxcox(column_positive)
    return transformed

# Aplicar transformación de Yeo-Johnson (puede manejar negativos)
def yeo_johnson_transform(column):
    transformed, _ = stats.yeojohnson(column)
    return transformed

# Revisar las columnas numéricas del DataFrame
numeric_cols = data_transformed.select_dtypes(include=['float64', 'int64']).columns

# Aplicar transformaciones a las columnas numéricas
for col in numeric_cols:
    # Seleccionar transformación adecuada según la distribución
    if (data_transformed[col] <= 0).any():
        # Usar Yeo-Johnson para datos con valores negativos o ceros
        data_transformed[col] = yeo_johnson_transform(data_transformed[col])
    else:
        # Usar Box-Cox para datos estrictamente positivos
        data_transformed[col] = boxcox_transform(data_transformed[col])

In [ ]:
data_transformed2 = data_transformed.copy()

### Modelo con 4 clusters

In [ ]:
pca_gmm_tra = PCA(n_components=0.90).fit(data_transformed)
pca_gmm_tra.n_components_

In [ ]:
pca_gmm_tra.explained_variance_ratio_

In [ ]:
pca_gmm_tra = PCA(n_components=3).fit(data_transformed)
X_pca_gmm_tra = pca_gmm.fit_transform(data_transformed)

In [ ]:
X_pca_gmm_tra2 = X_pca_gmm_tra.copy()

In [ ]:
# Modelo GMM- entrenamiento del modelo con 4 componentes con una covarianza tipo full
model_gmm_tra_4 = GaussianMixture(n_components=4, random_state=123, covariance_type = 'full').fit(X_pca_gmm_tra)

In [ ]:
# Predicciones y evaluación del modelo GMM con las metricas silhouette_score y calinski
labels_ = model_gmm_tra_4.predict(X_pca_gmm_tra)

print(" ### GMMM ###")
print('Silhouette Score: ', silhouette_score(X_pca_gmm_tra, labels_))
print('Calinski harabasz score: ', calinski_harabasz_score(X_pca_gmm_tra, labels_))

# Selección del modelo

In [ ]:
# import pandas as pd

# # Inicializamos una lista vacía para almacenar los resultados
# resultados = []

# # --- Evaluación del modelo kmeans sin reduccion con 4 clusters---
# resultados.append({
#     'Modelo': 'KMeans (sin reducción)',
#     'Clusters': 4,
#     'Inertia': model_kmeans_4.inertia_,
#     'Silhouette Score': silhouette_score(data_aleatorio1, model_kmeans_4.labels_),
#     'Calinski-Harabasz Score': calinski_harabasz_score(data_aleatorio1, model_kmeans_4.labels_)
# })

# # --- Evaluación del modelo kmeans sin reduccion con 8 clusters---
# resultados.append({
#     'Modelo': 'KMeans (sin reducción)',
#     'Clusters': 8,
#     'Inertia': model_kmeans_8.inertia_,
#     'Silhouette Score': silhouette_score(data_aleatorio2, model_kmeans_8.labels_),
#     'Calinski-Harabasz Score': calinski_harabasz_score(data_aleatorio2, model_kmeans_8.labels_)
# })

# # --- Evaluación del modelo kmeans 4 clusters con dataset reducido---
# resultados.append({
#     'Modelo': 'KMeans (PCA)',
#     'Clusters': 4,
#     'Inertia': model_kmeans_pca_4.inertia_,
#     'Silhouette Score': silhouette_score(X_pca, model_kmeans_pca_4.labels_),
#     'Calinski-Harabasz Score': calinski_harabasz_score(X_pca, model_kmeans_pca_4.labels_)
# })

# # --- Evaluación del modelo kmeans 6 clusters con dataset reducido---
# resultados.append({
#     'Modelo': 'KMeans (PCA)',
#     'Clusters': 6,
#     'Inertia': model_kmeans_pca_6.inertia_,
#     'Silhouette Score': silhouette_score(X_pca2, model_kmeans_pca_6.labels_),
#     'Calinski-Harabasz Score': calinski_harabasz_score(X_pca2, model_kmeans_pca_6.labels_)
# })

# # --- Evaluación del modelo jerárquico ---
# # Solo añadimos los coeficientes de Silhouette y Calinski-Harabasz para el modelo jerárquico con 3 clusters
# for clusters, silhouette, calinski in zip(x_range, silhouette_scores_hc, calinski_scores_hc):
#     if clusters == 3:  # Filtramos para el caso de 3 clusters
#         resultados.append({
#             'Modelo': 'Jerárquico',
#             'Clusters': clusters,
#             'Inertia': None,  # Jerárquico no tiene inertia
#             'Silhouette Score': silhouette,
#             'Calinski-Harabasz Score': calinski
#         })


# # --- Evaluación del modelo Gaussian Mixture Clustering ---
# resultados.append({
#     'Modelo': 'Gaussian Mixture',
#     'Clusters': len(set(labels_)),  # Asumiendo que los labels_ ya están generados
#     'Inertia': None,  # GMM no tiene inertia
#     'Silhouette Score': silhouette_score(df_new_scaled, labels_),
#     'Calinski-Harabasz Score': calinski_harabasz_score(df_new_scaled, labels_)
# })

# # Convertir la lista de resultados en un DataFrame
# df_resultados = pd.DataFrame(resultados)

# # Mostrar el DataFrame
# df_resultados


In [ ]:
import pandas as pd

# Inicializamos una lista vacía para almacenar los resultados
resultados = []

# --- Evaluación del modelo kmeans sin reduccion con 4 clusters---
resultados.append({
    'Modelo': 'KMeans (sin reducción)',
    'Clusters': 4,
    'Inertia': model_kmeans_4.inertia_,
    'Silhouette Score': silhouette_score(data_aleatorio1, model_kmeans_4.labels_),
    'Calinski-Harabasz Score': calinski_harabasz_score(data_aleatorio1, model_kmeans_4.labels_)
})

# --- Evaluación del modelo kmeans sin reduccion con 8 clusters---
resultados.append({
    'Modelo': 'KMeans (sin reducción)',
    'Clusters': 8,
    'Inertia': model_kmeans_8.inertia_,
    'Silhouette Score': silhouette_score(data_aleatorio2, model_kmeans_8.labels_),
    'Calinski-Harabasz Score': calinski_harabasz_score(data_aleatorio2, model_kmeans_8.labels_)
})

# --- Evaluación del modelo kmeans 4 clusters con dataset reducido---
resultados.append({
    'Modelo': 'KMeans (PCA)',
    'Clusters': 4,
    'Inertia': model_kmeans_pca_4.inertia_,
    'Silhouette Score': silhouette_score(X_pca, model_kmeans_pca_4.labels_),
    'Calinski-Harabasz Score': calinski_harabasz_score(X_pca, model_kmeans_pca_4.labels_)
})

# --- Evaluación del modelo kmeans 6 clusters con dataset reducido---
resultados.append({
    'Modelo': 'KMeans (PCA)',
    'Clusters': 6,
    'Inertia': model_kmeans_pca_6.inertia_,
    'Silhouette Score': silhouette_score(X_pca2, model_kmeans_pca_6.labels_),
    'Calinski-Harabasz Score': calinski_harabasz_score(X_pca2, model_kmeans_pca_6.labels_)
})

# --- Evaluación del modelo jerárquico ---
# Solo añadimos los coeficientes de Silhouette y Calinski-Harabasz para el modelo jerárquico con 3 clusters
for clusters, silhouette, calinski in zip(x_range, silhouette_scores_hc, calinski_scores_hc):
    if clusters == 3:  # Filtramos para el caso de 3 clusters
        resultados.append({
            'Modelo': 'Jerárquico',
            'Clusters': clusters,
            'Inertia': None,  # Jerárquico no tiene inertia
            'Silhouette Score': silhouette,
            'Calinski-Harabasz Score': calinski
        })


# --- Evaluación del modelo Gaussian Mixture Clustering 4 clusters ---
resultados.append({
    'Modelo': 'Gaussian Mixture (4 clusters)',
    'Clusters': 4,
    'Inertia': None,  # GMM no tiene inertia
    'Silhouette Score': silhouette_score(X_pca_gmm, model_gmm_4.predict(X_pca_gmm)),
    'Calinski-Harabasz Score': calinski_harabasz_score(X_pca_gmm, model_gmm_4.predict(X_pca_gmm))
})

# --- Evaluación del modelo Gaussian Mixture Clustering 5 clusters ---
resultados.append({
    'Modelo': 'Gaussian Mixture (5 clusters)',
    'Clusters': 5,
    'Inertia': None,  # GMM no tiene inertia
    'Silhouette Score': silhouette_score(X_pca_gmm2, model_gmm_5.predict(X_pca_gmm2)),
    'Calinski-Harabasz Score': calinski_harabasz_score(X_pca_gmm2, model_gmm_5.predict(X_pca_gmm2))
})

# --- Evaluación del modelo Gaussian Mixture Clustering 4 clusters con transformación ---
resultados.append({
    'Modelo': 'Gaussian Mixture (PCA)',
    'Clusters': 4,
    'Inertia': None,  # GMM no tiene inertia
    'Silhouette Score': silhouette_score(X_pca_gmm_tra, model_gmm_tra_4.predict(X_pca_gmm_tra)),
    'Calinski-Harabasz Score': calinski_harabasz_score(X_pca_gmm_tra, model_gmm_tra_4.predict(X_pca_gmm_tra))
})


# Convertir la lista de resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Mostrar el DataFrame
df_resultados

In [ ]:
# Encontrar el modelo con el mejor Silhouette Score
mejor_modelo_silhouette = df_resultados.loc[df_resultados['Silhouette Score'].idxmax()]

# Mostrar el mejor modelo con sus coeficientes
print("\nModelo con el mejor Silhouette Score y sus coeficientes:")
print(mejor_modelo_silhouette)

In [ ]:
# Realizamos una copia aleatoria de 15000 datos en el Dataframe original para agregar los clusters

df_shc2_ale = df_shc2.sample(n=15000, random_state=123)
df_shc2_ale.head()

In [ ]:
# Crear una nueva columna en el DataFrame original con las etiquetas de los clusters (4)

labels_kmeans = model_kmeans_4.labels_

df_shc2_ale['Cluster'] = labels_kmeans

# Imprimir el DataFrame con las etiquetas de los clusters
print(df_shc2_ale)

In [ ]:
grupos = df_shc2_ale.groupby('Cluster')

# Iterar sobre cada cluster y mostrar la descripción por separado
for cluster, grupo in grupos:
    print(f"### Descripción para el Cluster {cluster} ###")
    print(grupo.describe())
    print("\n")  # Línea vacía para mejor separación entre tablas

In [ ]:
# Conteo por tipo de cluster
cluster_counts = df_shc2_ale['Cluster'].value_counts()

print(cluster_counts)

## Descripción detallada de los Clusters


Cluster 0:
Este grupo se caracteriza por vehículos con un tamaño de motor intermedio (promedio de 2.97L), un kilometraje moderado (90,957 km) y un precio intermedio (£14,564). Los autos en este cluster tienen una antigüedad relativamente baja (6.8 años), lo que los convierte en una opción atractiva para compradores que buscan un buen balance entre calidad y precio. Estos vehículos son ideales para aquellos que priorizan la relación costo-beneficio, ofreciendo autos en buen estado, a un precio accesible, sin ser de lujo.

Cluster 1:
Los vehículos de este cluster son los más premium, con menor kilometraje (45,465 km), mayor antigüedad (14.45 años) y precios elevados (£37,062). Estos autos parecen mantener su valor a lo largo del tiempo, lo que los hace atractivos para compradores que valoran la calidad y durabilidad, independientemente de la antigüedad del vehículo. Este grupo se compone principalmente de autos de alta gama, bien conservados, que ofrecen confianza y robustez, y son una opción ideal para compradores que buscan autos con una mayor vida útil y menor desgaste.

Cluster 2:
Este cluster agrupa vehículos con motores grandes (3.03L), alto kilometraje (154,833 km) y precios elevados (£37,161), a pesar de la considerable cantidad de kilómetros recorridos. Aun así, los autos de este grupo logran mantener un precio elevado debido a su robustez y tamaño de motor, lo que sugiere que son valorados por su rendimiento a largo plazo. Estos autos son ideales para compradores que buscan vehículos potentes, sin importar el kilometraje, y que valoran características como el tamaño del motor y la capacidad de mantener su valor a lo largo del tiempo.

Cluster 3:
Este grupo representa la opción más accesible en términos de precio, con un promedio de £13,582. Los vehículos en este cluster son más antiguos (19.3 años) y tienen un kilometraje elevado (110,215 km). Estos autos atraen a compradores con presupuestos ajustados que buscan vehículos funcionales a bajo costo, sin preocuparse tanto por el desgaste o la antigüedad. Es una opción perfecta para aquellos que priorizan el precio por encima de otras características y que buscan un vehículo que cumpla su propósito básico.

## Conclusiones

Los resultados muestran que el kilometraje y la antigüedad son factores determinantes en la valoración de los vehículos, pero también destacan otros aspectos como el tamaño del motor y la calidad percibida. Mientras que los vehículos premium logran mantener su valor con el paso del tiempo (Clusters 1 y 2), los autos más accesibles (Clusters 0 y 3) brindan opciones adecuadas para aquellos que buscan equilibrio entre costo y funcionalidad. Esta segmentación proporciona una herramienta útil para adaptar las estrategias de mercado y optimizar la oferta de vehículos según las distintas necesidades y prioridades de los compradores, permitiendo atender tanto a quienes buscan lujo y durabilidad como a aquellos que valoran la asequibilidad.

En términos de ventas, los vehículos del Cluster 0, que suman 3,850 unidades, fueron los más vendidos, lo que indica una fuerte demanda por autos de menor costo que cumplen con las expectativas básicas de funcionalidad. Los Cluster 2 y Cluster 1, con 3,734 y 3,728 vehículos vendidos respectivamente, también muestran un buen desempeño, reflejando el interés de los compradores en vehículos que ofrecen una buena relación calidad-precio y calidad premium. Por último, los vehículos del Cluster 3, con 3,688 unidades vendidas, se posicionan como opciones intermedias, atrayendo a quienes buscan un balance entre precio y características deseables. Esta distribución de ventas resalta la diversidad en las preferencias de los consumidores y la importancia de ofrecer una gama de opciones que se ajusten a diferentes segmentos del mercado.